In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from pydantic import BaseModel
from datetime import datetime
from pprint import pprint

In [ ]:
class Scene(BaseModel):
    name: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]

class Scenes(BaseModel):
    scenes: list[Scene]

scenes_list = ["출퇴근길", "근무중", "게임", "AI 공부", "애완견 돌보기", "휴식"]

'출퇴근길, 근무중, 게임, AI 공부, 애완견 돌보기, 휴식'

In [4]:
class User(BaseModel):
    name: str
    location: str
    birthdate: datetime
    occupation: str
    personality: list[str]
    scenes: list[Scene] = []
    prompt: str
    positives: list[str]
    negatives: list[str]
    
user_context = {
    'name': '윤형석',
    'location': '서울',
    'birthdate': datetime.strptime('1990-03-28', '%Y-%m-%d'),
    'occupation': '회사원',
    'personality': ['Extroverted', 'Intuitive', 'Thinking', 'Perceiving'],
    'prompt': 'Your prompt here',
    'positives': ['사고력', '지적 호기심', '창의력'],
    'negatives': ['불안', '스트레스', '피곤']
}

user = User(**user_context)

user_bio = user.model_dump_json()

In [12]:
pydantic_parser = PydanticOutputParser(pydantic_object=Scenes)

scene_parser = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)

scene_prompt = ChatPromptTemplate.from_template("""
다음은 사용자의 하루를 구성하는 장면들입니다. 사용자의 정보와 이 장면들을 바탕으로, 각각의 장면에 어울리는 시간, 공간, 기타 태그를 3~5개씩 필요에 따라 작성하고, 아래 지침에 따라 json 포맷으로 반환하세요.
태그를 붙이는 목적은 할 일을 관리하기 위한 데이터베이스에 사용하기 위해서입니다. 각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.

시간 태그에는 휴일 여부, 요일, 하루 중의 시간대 등의 정보를 포함하세요.
공간 태그에는 사용자의 위치, 활동하는 장소 등의 정보를 포함하세요.
기타 태그에는 시간과 공간 태그에 포함되지 않지만 할 일의 맥락과 상황을 검색하기에 좋은 정보를 포함하세요.

사용자 정보: {bio}
장면: {scenes}
지침: {format_instruction}
""").partial(format_instruction=pydantic_parser.get_format_instructions())

scenes_chain = scene_prompt | scene_parser | pydantic_parser
response = scenes_chain.invoke({"bio": user_bio, "scenes": scenes_list})

for scene in response.scenes:
    user.scenes.append(scene)
    print(scene.model_dump_json())

{"name":"출퇴근길","location_tags":["서울","지하철","버스","도로"],"time_tags":["주중","오전","오후"],"other_tags":["교통","스트레스","바쁜 일정"]}
{"name":"근무중","location_tags":["사무실","서울"],"time_tags":["주중","오전","오후"],"other_tags":["업무","집중","스트레스","팀워크"]}
{"name":"게임","location_tags":["집","서울"],"time_tags":["주말","저녁"],"other_tags":["취미","여가","스트레스 해소","사회적 활동"]}
{"name":"AI 공부","location_tags":["집","서울"],"time_tags":["주말","오후"],"other_tags":["학습","지적 호기심","자기계발","미래 기술"]}
{"name":"애완견 돌보기","location_tags":["집","서울","공원"],"time_tags":["주말","오후"],"other_tags":["애완동물","스트레스 해소","책임감","운동"]}
{"name":"휴식","location_tags":["집","서울"],"time_tags":["주말","저녁"],"other_tags":["휴식","재충전","스트레스 해소","자기 돌봄"]}


In [6]:
import json

user_bio = user.model_dump_json()
user_dict = json.loads(user_bio)
user_dict['scenes'][4]

{'name': '애완견 돌보기',
 'location_tags': ['집', '서울', '공원'],
 'time_tags': ['주말', '오후'],
 'other_tags': ['애완동물', '스트레스 해소', '책임감']}

In [7]:
# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=None)

In [8]:
from langchain.schema import BaseOutputParser

class CustomListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> list[str]:
        responses = text.split("---")
        items = [response.strip() for response in responses]
        return items

    def get_format_instructions(self) -> str:
        return '서로 다른 답변은 "---"로 구분하세요.'

# 파서 초기화
response_parser = CustomListOutputParser()

In [9]:
prompt_template = ChatPromptTemplate.from_template("""
다음은 사용자 정보입니다. 이 정보를 바탕으로, 사용자의 성격과 하루 일과, 주요 관심사를를 상상해서 1문단으로 작성하세요.
이를 작성하는 이유는 사용자의 할 일을 사용자의 생활패턴과 맥락에 맞게 구체화하여 추천하기 위해서입니다.

작성된 내용 중 사용자가 적합한 것을 선택할 수 있도록, 서로 다른 내용의 답변을 3~5개 생성하세요.
각각의 답변은 사용자 정보의 다른 부분에 집중하며, 서로 비슷하지 않은 내용이어야 합니다.
예를 들어 한 답변이 "대중교통"이라는 키워드에 집중한다면, 다른 답변은 "도서관" 등 다른 맥락에 집중할 수 있습니다.
만약 비슷한 답변이 생성된다면 생략하세요.

{format_instruction}

사용자 정보: {bio}

답변:
""").partial(format_instruction=response_parser.get_format_instructions())

chain = prompt_template | llm | response_parser
responses = chain.invoke({"bio": user_bio})

In [10]:
for response in responses:
    pprint(response)
    print("---")

('윤형석은 서울에서 회사원으로 일하며, 외향적이고 직관적인 성격을 가진 사람입니다. 그는 주중에는 출퇴근길에 지하철이나 버스를 이용하며, '
 '이 시간 동안 스트레스를 느끼기도 하지만, 사람들과의 소통을 즐깁니다. 근무 중에는 집중력을 발휘해 업무를 처리하며, 주말에는 집에서 '
 '게임을 하거나 AI 공부를 통해 지적 호기심을 충족시키는 것을 좋아합니다. 또한, 애완견을 돌보는 시간을 통해 책임감을 느끼고 스트레스를 '
 '해소하며, 저녁에는 휴식을 취하며 재충전하는 일상을 보냅니다.')
---
('윤형석은 주중에 출퇴근길에 지하철을 이용하며, 이 시간 동안 다양한 사람들과의 소통을 즐깁니다. 그는 외향적인 성격 덕분에 대중교통에서의 '
 '소소한 대화나 만남을 통해 하루를 시작하는 것을 좋아합니다. 이러한 경험은 그에게 스트레스를 줄이는데 도움을 주며, 업무에 대한 집중력을 '
 '높이는 데 기여합니다.')
---
('주말에는 윤형석이 집에서 AI 공부를 하며 지적 호기심을 충족시키는 시간을 가집니다. 그는 새로운 기술과 트렌드에 대한 학습을 통해 '
 '자기계발에 힘쓰며, 이러한 과정에서 느끼는 성취감은 그에게 큰 만족을 줍니다. 이 시간은 그가 스트레스를 해소하고, 창의력을 발휘할 수 '
 '있는 기회를 제공합니다.')
---
('윤형석은 주말 오후에 애완견을 돌보는 시간을 소중히 여깁니다. 공원에서 애완견과 함께 산책하며 자연을 느끼고, 책임감을 느끼는 이 시간은 '
 '그에게 큰 스트레스 해소가 됩니다. 그는 애완견과의 교감을 통해 정서적인 안정감을 찾고, 일상에서의 피로를 잊을 수 있습니다.')
---
('주말 저녁에는 윤형석이 집에서 휴식을 취하며 재충전하는 시간을 가집니다. 그는 이 시간을 통해 일주일 동안 쌓인 스트레스를 해소하고, '
 '다음 주를 준비하는 데 필요한 에너지를 충전합니다. 편안한 음악을 듣거나 좋아하는 영화를 보며 여유를 즐기는 것이 그의 일상에서 중요한 '
 '부분입니다.')
---


In [11]:
def set_prompt(user: User, responses: list[str], index: int):
    if index < 0 or index >= len(responses):
        raise IndexError("Index out of bounds.")
    user.prompt = responses[index].strip()

set_prompt(user, responses, 0)
user.prompt

'윤형석은 서울에서 회사원으로 일하며, 외향적이고 직관적인 성격을 가진 사람입니다. 그는 주중에는 출퇴근길에 지하철이나 버스를 이용하며, 이 시간 동안 스트레스를 느끼기도 하지만, 사람들과의 소통을 즐깁니다. 근무 중에는 집중력을 발휘해 업무를 처리하며, 주말에는 집에서 게임을 하거나 AI 공부를 통해 지적 호기심을 충족시키는 것을 좋아합니다. 또한, 애완견을 돌보는 시간을 통해 책임감을 느끼고 스트레스를 해소하며, 저녁에는 휴식을 취하며 재충전하는 일상을 보냅니다.'